In [3]:
import pandas as pd 
import numpy as np

In [4]:
#Set up fake data for individuals 
#age: the age of individual 
#sex: the sex of individual F:Female M:Male 
#Energydrink_consumption:Energy_drink consumption per week 

#Create list
IND = {"age":["59","54","35","73","49","30"],
      "sex":["F","F","F","F","F","F"],
      "Energydrink_consumption":["3","4","5","6","7","1"]}

#Create dataframe 
IND = pd.DataFrame(IND)

#Let age to become a numerical value
IND["age"] = IND["age"].apply(pd.to_numeric)

print(IND)

   age sex Energydrink_consumption
0   59   F                       3
1   54   F                       4
2   35   F                       5
3   73   F                       6
4   49   F                       7
5   30   F                       1


In [5]:
#Create aggreagted lists 

#Create emptylist based on constraints 
Under50 = []
Over50 = []
M = []
F = []

#For loop for each constraints -> binary outputs
for i in range(len(IND)):
    
    if IND.loc[i,"age"] < 50: 
            Under50.append(1)
            Over50.append(0)

    elif IND.loc[i,"age"] > 50: 
            Under50.append(0)
            Over50.append(1)

for i in range(len(IND)):

    if IND.loc[i,"sex"] == "M": 
            M.append(1)
            F.append(0)
            
    elif IND.loc[i,"sex"] == "F": 
            M.append(0)
            F.append(1)
            
        
#Combine all lists 
IND_Agg =[Under50,Over50,M,F]

print(IND_Agg)


[[0, 0, 1, 0, 1, 1], [1, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1]]


In [6]:
#set up fake constraints 

#CONS = {"Under50":["8"],"Over50": ["4"],"M" : ["6"],"F" : ["6"]}
#CONS = pd.DataFrame(CONS) 
#for col in CONS: 
#    CONS[col] = CONS[col].apply(pd.to_numeric)

CONS = [[8,4,2,10],[3,3,0,6],[8000,4000,2000,10000]]

print(CONS)

[[8, 4, 2, 10], [3, 3, 0, 6], [8000, 4000, 2000, 10000]]


In [47]:
#Main IPF Algorithmn

#weight = [1,1,1,1,1,1]

def IPF(X,Y):
    
    weight = [1]*len(X[0])

    for k in range(len(Y)):
        
        X_new = np.multiply(weight,X[k])
        
        for i in range(len(X[k])):
            
            if X[k][i] == 1:
                
                weight[i] = weight[i]*Y[k]/sum(X_new)
                
            else: 
                
                weight[i] = weight[i]
                
    #return(weight)
    
    #print("Iteration " + str(k))
    
    TAE = Y[k] - sum(X_new) 
    
    return(TAE,weight)
    #print(X_new)
   
   
            

In [36]:
# Create permutations function of a given list -> Perhaps use the build in function from the iterativetools -> this basically calculates all the combinations 
def permutation(lst): 
  
    # If lst is empty then there are no permutations 
    if len(lst) == 0: 
        return [] 
  
    # If there is only one element in lst then, only 
    # one permuatation is possible 
    if len(lst) == 1: 
        return [lst] 
  
    # Find the permutations for lst if there are 
    # more than 1 characters 
  
    l = [] # empty list that will store current permutation 
  
    # Iterate the input(lst) and calculate the permutation 
    for i in range(len(lst)): 
        
       m = lst[i] 
  
       # Extract lst[i] or m from the list.  remLst is 
       # remaining list 
       remLst = lst[:i] + lst[i+1:] 
  
       # Generating all permutations where m is first 
       # element 
       for p in permutation(remLst): 
           l.append([m] + p) 
    return l 

In [37]:
#IPF2 - iteration all possible permutations -> find the best iteration with the smallest TAE

def IPF2(X,Y,max_iteration = 1):
    
    #Number of iteration for each permutation order 
    
    iteration = 0 
    
    smallest = 1000000
    
    smallest_iteration = 11
    
    #Generate all possible permutation for individual data 
    perm = permutation(X)
    
    #Generate all possible permutation for Constraint data 
    perm2 = permutation(Y)
    
    #While statement that loop through all iteration till it achieves max_iteration
    while iteration < max_iteration: 
        
        #Generate new individual data for each iteration 
        #For example, the first iteration has the following order: M, <50, F, >50 
        X_new = perm[iteration]

        #Generate new Constraints data for each iteration 
        #For example, the first iteration has the following order: M, <50, F, >50
        Y_new = perm2[iteration]
        
        #Note: The variables order of constraint and individual should be the same
        
        
        print("Iteration: " + str(iteration))
        
        Result = IPF(X_new,Y_new)
        
        TAE = Result[0]
        
        Weight = Result[1]
        #print(TAE)
        
        iteration += 1 
        
        if smallest > abs(TAE):
            
            smallest = abs(TAE)
            
            smallest_iteration = iteration 
            
            final_weight = Weight
            
    print("Converged! Smallest TAE: " + str(smallest) + " " + "at iteration: " + str(smallest_iteration))

    return(final_weight)
    
            
#IPF2(IND_Agg,CONS,max_iteration = 24) 

In [38]:
#Return a complete weight matrix for each constraints and for each area based on the iteration with the smallest TAE
def IPF3(X,Y,Z = 1):
    
    area_weight = []
    
    for i in range(len(Y)):
        
        weight = IPF2(X,Y[i],max_iteration = Z)
        
        area_weight.append(weight)
        
    return(area_weight)
        

#a =  IPF3(IND_Agg,CONS,24)
#print(a)

In [39]:
#Testing the new algorithm on real data

#import all data 
ind = pd.read_csv("C:/Users/44749/Desktop/IterativeProportionalFitting-Test/ind.csv",index_col=False) 
cons = pd.read_csv("C:/Users/44749/Desktop/IterativeProportionalFitting-Test/cons.csv") 

ind_list = ind.iloc[:,1:].values.tolist()
cons_list = cons.iloc[:,1:].values.tolist()
#len(ind_list[0])
#len(cons_list[0])


In [40]:
#Reformat individual data 
newlist = []
for x in range(len(cons_list[0])):
    
    innerlist = []
    
    for y in range(len(ind_list)):
        
        innerlist.append(ind_list[y][x])
        
    newlist.append(innerlist)

len(newlist)

13

In [85]:
#Performing 1st IPF algorithm on the data 

area_weight = []

widgets = ['Test: ', Percentage(), ' ', Bar(marker=RotatingMarker()),
               ' ', ETA(), ' ', FileTransferSpeed()]

pbar = ProgressBar(widgets=widgets,maxval=len(cons_list)).start()

for i in range(len(cons_list)):
    
    result = IPF(newlist,cons_list[i])
    
    area_weight.append(result[1])
    
    pbar.update(i+1)

pbar.finish()




Test: 100% |||||||||||||||||||||||||||||||||||||||||| Time: 1:26:33   0.06  B/s


In [1]:
#area_weight

In [2]:
pip ProgressBar

ERROR: unknown command "ProgressBar"
Note: you may need to restart the kernel to use updated packages.
